In [1]:
'''''
{
"title": "15_CreateLocaInfoDF",
"keywords": "15_CreateLocaInfoDF, CreateLocalInfo",
"categories": "",
"description": "Hier die Implementierungsnotizen zu der Erstellung der Info-Datei",
"level": "120",
"pageID": "07112020200718-15_CreateLocaInfoDF"
}
'''''

'\'\'\n{\n"title": "15_CreateLocaInfoDF",\n"keywords": "15_CreateLocaInfoDF, CreateLocalInfo",\n"categories": "",\n"description": "Hier die Implementierungsnotizen zu der Erstellung der Info-Datei",\n"level": "120",\n"pageID": "07112020200718-15_CreateLocaInfoDF"\n}\n'

<h1>Erstellen einer lokalen Info-Datei</h1>

Dev-ID: 15_CreateLocaInfoDF

# Ziel 
Mit diesem Skript soll eine lokale Info-Datei im CSV Format enstehen, welche als weiter führende Grundlage für die bessere Struktuierung meiner Website dienen soll. Konkret soll ein Pandas-DF entstehen und lokal gespeichert werden welche folgende Informationen zu jedem HTML-File enthält:

- [x]  pageid
- [x]  Absoluter Pfad lokal
- [x]  Relativer Pfad Remote
- [x]  FielCreationDate
- [x]  FileModifiedDate
- [x]  FileSize
- [x]  level für die Hierarchie
- [x]  Keywords = Tags
- [x]  categories=Manuell
- [x]  description
- [x]  categoriesList = Automatisch von Directory Struktuierung
- [x]  section = höchste Categorie
- [x]  sectionPathAbsolute
- [x]  sectionPathRelative
- [x]  Remote-Adresse = webadresse der Seite
- [x]  Vorgelagerter Page-ID = File das im gleichen Ordner liegt aber ein kleineres Level hat
- [x]  nachgelagerte Page-ID = File das im gleichen Ordner liegt aber ein größeres Level hat




# Vorbedingung
- Es müssen die Content-Seiten mit einem Frame-Matter ausgestattet sein und dort müssen folgende Informationen 
    - gepflegt sein:
        - pageID
        - level

# Anschluss
- Es müssen die Content-Seiten mit einem Frame-Matter ausgestattet sein und dort müssen folgende Informationen 
    - gepflegt sein:
        - pageID
        - level

# Technologien
- Python

# Status der Idee
- [x]  Die Iddee exisitiert & Die Ideenentwicklung wurde gestarte
- [x]  Implementierung und die Technologie bekannt & dokumentiert
- [x]  Git Checkout-Developmentbranch 
- [x]  Implementierung der Technologie in diesem Notebook
- [x]  Implementierung der Technologie in den Produktivbereich = *.py Erstellt und in den Publish Prozess implementiert
- [x]  Test Local 
- [x]  Test Remote 
- [x]  Idee abgeschlossens

# Umsetzung
Ich iterier hierbei durch das "content", da hierbei weniger Transfomrationen ausgeführt werden müssen und die resultierend Info dennoch gleich ist.

In [2]:
from glob import glob
import os
import shutil
import sys
from markdown2 import markdown 
import re
import json
from json import load
import pandas as pd
import os.path
import time

In [3]:
ErrorMessage=""
dir_path = "C:/DocTool/"
completeList = []
for root, dirs, files in os.walk(dir_path+"content/"):
    for file in files:
        currentRow=[]
        if file.endswith(".md"):
            sourceAbsolute = (os.path.join(root, file)).replace("\\", "/")
            sourceRelative = sourceAbsolute.replace("C:/DocTool/content", "")
            fileCreationTime = time.ctime(os.path.getmtime(sourceAbsolute))
            fileLastModifiedTime = time.ctime(os.path.getctime(sourceAbsolute))
            filesize = os.path.getsize(sourceAbsolute)

            with open(sourceAbsolute, encoding="utf8") as markdown_file:
                    article = markdown(
                        markdown_file.read())
                    
                    metadata = {}
                    if("'''''" in article):
                        splited = re.split("'''''", article)
                        metadata = (splited[1])
                        metadata = metadata.replace("<p>", "")
                        metadata = metadata.replace("</p>", "")
                        metadata = metadata.strip()
                        metadata=json.loads(metadata)
                        article = splited[2]
            try:
                pageID=metadata["pageID"]
            except:
                pageID=""
                ErrorMessage = ErrorMessage + "Die Website: \n" + str(sourceAbsolute)+ "\nhat keine Page-ID \n\n"
            
            try:
                keywords=metadata["keywords"]
            except:
                keywords=""
                ErrorMessage = ErrorMessage + "Die Website: \n" + str(sourceAbsolute)+ "\nhat keine keywords \n\n"

            try:
                categories=metadata["categories"]
            except:
                categories=""
                ErrorMessage = ErrorMessage + "Die Website: \n" + str(sourceAbsolute)+ "\nhat keine categories \n\n"

            try:
                level=metadata["level"]
            except:
                level=""
                ErrorMessage = ErrorMessage + "Die Website: \n" + str(sourceAbsolute)+ "\nhat keine level \n\n"

            try:
                description=metadata["description"]
            except:
                description=""
                ErrorMessage = ErrorMessage + "Die Website: \n" + str(sourceAbsolute)+ "\nhat keine description \n\n"


            currentRow.append(pageID)
            currentRow.append(sourceAbsolute)
            currentRow.append(sourceRelative)
            currentRow.append(fileCreationTime)
            currentRow.append(fileLastModifiedTime)
            currentRow.append(filesize)
            currentRow.append(level)
            currentRow.append(keywords)
            currentRow.append(categories)
            currentRow.append(description)
            completeList.append(currentRow)

            # Ablegen in df


file = open("./ErrorList-CreateLocalInfoFile.txt", "w", encoding="utf-8")
file.write(ErrorMessage)
file.close()

In [4]:
def retrungCategoriesList(x):
    '''
    Extrahiere die Kategorie Informaitonen vondem pfad und gebe eine Liste zurück
    '''
    x = x.replace("C:/DocTool/content/", "")
    x = re.split("/", x)[:-1]
    return(x)

def returnWebAdresse(x):
    '''
    Extrahiere die Pfade, sodass die Webadresse ableitbar ist
    '''
    x = x.replace("C:/DocTool/content/", "https://jhc90.github.io/")
    x = x.replace(".md", ".html")
    return(x)

def retrunSection(x):
    '''
    Extrahiere die Pfade, sodass die Webadresse ableitbar ist
    '''
    if(len(x) == 0):
        x = x
    else:
        x = x[-1]
    return(x)

def transformCSVsToList(x):
    '''
    Extrahiere die Pfade, sodass die Webadresse ableitbar ist
    '''
    x = re.split(",", x)
    return(x)

def returnSectionPathAbsolute(x):
    '''
    Extrahiere die absoluten Pfade für die Sections
    '''
    x = re.split("/", x)[:-1]
    y=""
    for element in x:
        y = y + element + "/"
    x = y
    return(x)

def returnSectionPathRelative(x):
    '''
    Extrahiere die absoluten Pfade für die Sections
    '''
    x = x.replace("C:/DocTool/content", "")
    if(x == ""):
        x = "/"
    return(x)

In [5]:


labels =["pageid", "fileNameAbsolut", "fileNameRelative", "fileLastModifiedTime", "fileCreationTime", "fileSizeInBytes", "level", "keywords", "categories", "description" ]

DF_MetaData = pd.DataFrame(data=completeList)
DF_MetaData.columns=labels
DF_MetaData['keywords'] = DF_MetaData['keywords'].apply(transformCSVsToList)
DF_MetaData['categories'] = DF_MetaData['categories'].apply(transformCSVsToList)
DF_MetaData['categoriesList'] = DF_MetaData['fileNameAbsolut'].apply(retrungCategoriesList)
DF_MetaData['section'] = DF_MetaData['categoriesList'].apply(retrunSection)
DF_MetaData['sectionPathAbsoulte'] = DF_MetaData['fileNameAbsolut'].apply(returnSectionPathAbsolute)
DF_MetaData['sectionPathRelative'] = DF_MetaData['sectionPathAbsoulte'].apply(returnSectionPathRelative)
DF_MetaData['webadress'] = DF_MetaData['fileNameAbsolut'].apply(returnWebAdresse)
DF_MetaData['vorgelagerteSectionPage'] = ""
DF_MetaData['nachgelagerteSectionPage'] = ""
DF_MetaData['sectionIndex'] = "https://jhc90.github.io/index.html"


# in diesem block werden nun die Level ausgelesen und in eine Reihenfolge gebracht, sodass die Sequent der Files angepasst ist
# Das ist wichtig für die vor und nachgelagerten Dateien für die Sections.
# DF_MetaData.head()
uniqueSectionPathes = DF_MetaData.sectionPathRelative.unique()


for element in uniqueSectionPathes:
    currentDF  = DF_MetaData.loc[DF_MetaData['sectionPathRelative'] == element]
    #print(currentDF.shape) => shape 12
    currentDFsorted = currentDF.sort_values(by='level')
    #print(currentDFsorted.shape)=> shape 12
    
    currentDFsorted = currentDFsorted.reset_index()
    currentDFsorted['level'] = currentDFsorted.index
    currentDFsorted = currentDFsorted.drop('index', axis=1)
    currentDFsorted['vorgelagerteSectionPage'] = currentDFsorted['pageid'].shift(periods=1)
    currentDFsorted['nachgelagerteSectionPage'] = currentDFsorted['pageid'].shift(periods=-1)
   
    DF_MetaData = DF_MetaData.append(currentDFsorted)
    DF_MetaData = DF_MetaData.drop_duplicates(subset=['fileNameAbsolut'], keep='last')
    
#print(DF_MetaData[['fileNameAbsolut', 'vorgelagerteSectionPage', 'nachgelagerteSectionPage']].head(50))
#DF_check = DF_MetaData[['fileNameAbsolut', 'sectionPathAbsoulte','vorgelagerteSectionPage', 'nachgelagerteSectionPage']]
#DF_check.to_csv('./ExcelCheck.csv',index=False)

# Speicher DF als CV lokal

DF_MetaData.to_csv('C:/DocTool/DocToolMaintainingScripts/SiteMetaInfos.csv',index=False)